In [ ]:
#テキストマイニング
#Excelの文字をすべて半角に
#=ASC(JIS(LEFT(D2,255)))&ASC(JIS(MID(D2,256,255)))

In [ ]:
wb_name = ".xlsx"
ws_name = ''

#対象のExcel列名
comment_column_name = ""
target_col = ""

In [ ]:
import pandas as pd
original_data = pd.read_excel(wb_name,sheet_name = ws_name)
#print(len(original_data))
#original_data.head()

In [ ]:
import MeCab
tagger = MeCab.Tagger()#入力文字にタグをつける解析器

In [ ]:
#除去したい言葉の決定
stop_words = ["%","%)","&","(",")","｡(","｡)","､｢","<",">","｢","｣","｣､","｣｢","?｣","?",")、","､","｡","･",",","-","~","~)","~:","一",".","/",":",";","±","+","=","Ⅱ","⇒","①","①【","②","､②","｡②","②【","③","､③","④","!","!!","(｢",")(","))","):",")｡",")､","､､","*","あ","い","こ","し","た","み","の","いき","こと","なり","よう","的"]

#追加
"""
｣､｢
"""

In [ ]:
#必要な品詞を登録
parts = ["名詞","動詞"]
not_noun_or_verb_but = ["なし"]

In [ ]:
keywords_groups_names = [
"1 ",
"2 ",
"3 ",
"4 ",
"5 ",
"6 ",
"7 ",
"8 ",
"9 ",
"10 "
]

#キーワード登録
category1 = [""]
category2 = [""]
category3 = [""]
category4 = [""]
category5 = [""]
category6 = [""]
category7 = [""]
category8 = [""]
category9 = [""]
category10 = [""]

In [ ]:
#全カテゴリーを一つのカテゴリーリストにする
keywords_groups = [category1,category2,category3,category4,category5,category6,category7,category8,category9,category10]

In [ ]:
#解析結果を記録するpandas表を作る
words_df = pd.DataFrame()

#カテゴリーを大きな表に加える
for j in range(len(keywords_groups_names)):
    words_df[keywords_groups_names[j]] = [0]
words_df.head()

In [ ]:
#大きな表
all_words_df= pd.DataFrame()

#Excelの行ごとに処理
for row in range(len(original_data)):#アンケートの行ごとに
    text = original_data[comment_column_name].iloc[row]#行ごとの自由記述の取得
    words = tagger.parse(text).splitlines()#自由記述を形態素解析
#抽出した単語を入れる表
    words_df = pd.DataFrame()#行ごとの単語を入れる表
#辞書情報を付けた単語ごとに
    for i in words:#解析した辞書情報付き単語毎に
        if i == "EOS" or i == "":
            continue#EOSと空欄は飛ばす
        word_tmp = i.split()[0]#単語を取得
        if word_tmp in not_noun_or_verb_but:
            words_df[word_tmp] = [1]#名詞と動詞以外の指定単語を一時リストに入れる
        part = i.split()[1].split(",")[0]#品詞を取得(確認済み)
        if not (part in parts):
            continue#登録した品詞以外ならfor i に戻る
        if word_tmp in stop_words:
            continue#登録した単語ならfor i に戻る
        words_df[word_tmp] = [1]#単語を一時リストに加えて１を足す
    all_words_df = pd.concat([all_words_df, words_df] ,ignore_index=True)#結果を大きな表に加える
print(len(all_words_df))
all_words_df.head()

In [ ]:
#大きな表
keywords_df= pd.DataFrame()

#Excelの行ごとに処理
for row in range(len(original_data)):#アンケートの行ごとに
    text = original_data[comment_column_name].iloc[row]#行ごとの自由記述の取得
    words = tagger.parse(text).splitlines()#自由記述を形態素解析
#抽出した単語を入れる表
    words_df = pd.DataFrame()#行ごとの単語を入れる
#元の表のカテゴリー順を守るための編集（詳細要確認
    for j in range(len(keywords_groups_names)):
        words_df[keywords_groups_names[j]] = [0]
#辞書情報を付けた単語ごとに
    for i in words:#解析した辞書情報付き単語毎に
        if i == "EOS" or i == "":
            continue#EOSと空欄は飛ばす
        word_tmp = i.split()[0]#単語を取得
#キーワード毎に
        for k in range(len(keywords_groups)):
            if word_tmp in keywords_groups[k]:
                words_df[keywords_groups_names[k]] = [1]#登録地名がある場合はアンケートの行に1を記入
    keywords_df = pd.concat([keywords_df, words_df] ,ignore_index=True)#結果を大きな表に加える
print(len(keywords_df))
keywords_df.head()

In [ ]:
all_words_df = pd.concat([keywords_df, all_words_df], axis=1)
all_words_df.head()

"""all_words_df = pd.merge(transfer_request_words_df,all_words_df,on="識別番号",how="right")
print(len(all_words_df))
all_words_df.head()"""

In [ ]:
all_words_df = all_words_df.fillna(0)
all_words_df.head()

In [ ]:
all_words_df = all_words_df.fillna(0)
all_words_df.head()

In [ ]:
Analysis_results = pd.concat([original_data, all_words_df],axis=1)#文章と単語解析を結合
Analysis_results.head()

In [ ]:
Analysis_results.to_csv("4元データと分析の結合.csv",encoding="utf_8_sig",index=True)